## GOAL: 
### 1. Identify topics amongs hotel reviews
### 2. For each hotel and each topic assign that hotel a score for that topic
### 3. Surface for each topic the most relevant sentences

In [1]:
### Import relevant modules

import nltk
nltk.download('stopwords')

# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

# re
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy
spacy.load('en')

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt

[nltk_data] Downloading package stopwords to /home/eric/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
### Import our data
happy = pd.read_csv("/media/eric/nachmanides/insight_data_science/resources/data_challenge/happy_hotels/hotel_happy_reviews.csv")
unhappy = pd.read_csv("/media/eric/nachmanides/insight_data_science/resources/data_challenge/happy_hotels/hotel_not_happy_reviews.csv")
all_reviews = pd.concat([happy,unhappy],axis=0)

### 1. Tokenize each sentence into a list of words, removing punctuation and unnecessary characters.

In [3]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuation

data_words = list(sent_to_words(all_reviews.Description))

print(data_words[:1])

[['stayed', 'here', 'with', 'husband', 'and', 'sons', 'on', 'the', 'way', 'to', 'an', 'alaska', 'cruise', 'we', 'all', 'loved', 'the', 'hotel', 'great', 'experience', 'ask', 'for', 'room', 'on', 'the', 'north', 'tower', 'facing', 'north', 'west', 'for', 'the', 'best', 'views', 'we', 'had', 'high', 'floor', 'with', 'stunning', 'view', 'of', 'the', 'needle', 'the', 'city', 'and', 'even', 'the', 'cruise', 'ships', 'we', 'ordered', 'room', 'service', 'for', 'dinner', 'so', 'we', 'could', 'enjoy', 'the', 'perfect', 'views', 'room', 'service', 'dinners', 'were', 'delicious', 'too', 'you', 'are', 'in', 'perfect', 'spot', 'to', 'walk', 'everywhere', 'so', 'enjoy', 'the', 'city', 'almost', 'forgot', 'heavenly', 'beds', 'were', 'heavenly', 'too']]


### 2. Build the bigram and trigram models

In [4]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['stayed', 'here', 'with', 'husband', 'and', 'sons', 'on', 'the', 'way', 'to', 'an', 'alaska_cruise', 'we', 'all', 'loved', 'the', 'hotel', 'great', 'experience', 'ask', 'for', 'room', 'on', 'the', 'north', 'tower', 'facing', 'north', 'west', 'for', 'the', 'best', 'views', 'we', 'had', 'high', 'floor', 'with', 'stunning', 'view', 'of', 'the', 'needle', 'the', 'city', 'and', 'even', 'the', 'cruise_ships', 'we', 'ordered', 'room', 'service', 'for', 'dinner', 'so', 'we', 'could', 'enjoy', 'the', 'perfect', 'views', 'room', 'service', 'dinners', 'were', 'delicious', 'too', 'you', 'are', 'in', 'perfect', 'spot', 'to', 'walk', 'everywhere', 'so', 'enjoy', 'the', 'city', 'almost', 'forgot', 'heavenly', 'beds', 'were', 'heavenly', 'too']


### 3. Remove Stopwords, Make Bigrams and Lemmatize

In [5]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [6]:
# Call the functions in order
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['stay', 'husband', 'son', 'alaska_cruise', 'love', 'hotel', 'great', 'experience', 'ask', 'face', 'good', 'view', 'high', 'floor', 'stunning', 'view', 'city', 'even', 'order', 'room', 'service', 'dinner', 'could', 'enjoy', 'perfect', 'view', 'room', 'service', 'dinner', 'delicious', 'perfect', 'spot', 'walk', 'everywhere', 'enjoy', 'city', 'almost', 'forget', 'heavenly', 'bed', 'heavenly']]


### 4. Create the Dictionary and Corpus needed for Topic Modeling

In [7]:
# The two main inputs to the LDA topic model are the dictionary(id2word) and the corpus. Let’s create them.

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 2), (5, 1), (6, 1), (7, 2), (8, 2), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 2), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 2), (24, 2), (25, 2), (26, 1), (27, 1), (28, 1), (29, 1), (30, 3), (31, 1)]]


In [8]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('alaska_cruise', 1),
  ('almost', 1),
  ('ask', 1),
  ('bed', 1),
  ('city', 2),
  ('could', 1),
  ('delicious', 1),
  ('dinner', 2),
  ('enjoy', 2),
  ('even', 1),
  ('everywhere', 1),
  ('experience', 1),
  ('face', 1),
  ('floor', 1),
  ('forget', 1),
  ('good', 1),
  ('great', 1),
  ('heavenly', 2),
  ('high', 1),
  ('hotel', 1),
  ('husband', 1),
  ('love', 1),
  ('order', 1),
  ('perfect', 2),
  ('room', 2),
  ('service', 2),
  ('son', 1),
  ('spot', 1),
  ('stay', 1),
  ('stunning', 1),
  ('view', 3),
  ('walk', 1)]]

### 5. Find the optimal number of topics for LDA
#### Build many LDA models with different values of number of topics (k) and pick the one that gives the highest coherence value. Choosing a ‘k’ that marks the end of a rapid growth of topic coherence usually offers meaningful and interpretable topics. 

In [21]:
### Define a function that  trains multiple LDA models and provides the models and 
### their corresponding coherence scores.

def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topics, 
                                           random_state=42,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [22]:
### Execute the function
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=2, limit=40, step=6)

KeyboardInterrupt: 

In [ ]:
# Graph coherence scores as a function of number of topics
limit=40; start=2; step=6;
x = range(start, limit, step)
plt.style.use("ggplot")
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [ ]:
# Select the model that gave the highest coherence value before flattening out, and print the topics
optimal_model = model_list[3]
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))

### 6. View the topics in the best LDA model

In [10]:
# Print the Keyword in the 10 topics
pprint(optimal_model.print_topics())
doc_lda = optimal_model[corpus]


# NOTES: 
# The weights reflect how important a keyword is to that topic.
# Looking at these keywords, we can guess what each topic is.

[(0,
  '0.107*"tiny" + 0.081*"level" + 0.060*"lounge" + 0.051*"one" + 0.048*"valet" '
  '+ 0.048*"certainly" + 0.047*"pull" + 0.046*"garage" + 0.041*"step" + '
  '0.035*"hall"'),
 (1,
  '0.097*"dark" + 0.073*"bedroom" + 0.065*"foot" + 0.056*"clearly" + '
  '0.055*"replace" + 0.053*"unit" + 0.045*"curtain" + 0.044*"outdate" + '
  '0.043*"dining" + 0.042*"concern"'),
 (2,
  '0.070*"month" + 0.068*"uncomfortable" + 0.055*"arrival" + 0.051*"decor" + '
  '0.049*"literally" + 0.043*"driver" + 0.042*"soon" + 0.042*"total" + '
  '0.040*"thought" + 0.039*"manage"'),
 (3,
  '0.125*"year" + 0.096*"always" + 0.058*"send" + 0.057*"home" + 0.050*"name" '
  '+ 0.047*"hope" + 0.045*"write" + 0.043*"wonderful" + 0.043*"love" + '
  '0.033*"resort"'),
 (4,
  '0.076*"bar" + 0.073*"wait" + 0.043*"move" + 0.039*"probably" + '
  '0.037*"weekend" + 0.036*"deal" + 0.032*"standard" + 0.032*"extra" + '
  '0.030*"low" + 0.029*"access"'),
 (5,
  '0.282*"stay" + 0.126*"staff" + 0.109*"clean" + 0.097*"location" + '


### 7. Compute Model Perplexity and Coherence Score

In [11]:
# Compute Perplexity
print('\nPerplexity: ', optimal_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=optimal_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -10.722577140998212

Coherence Score:  0.26429871586107045


### 8. Visualize the topics-keywords

In [14]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(optimal_model, corpus, id2word, mds='mmds')
vis

#### NOTES: 
##### 1. Each bubble on the left-hand side plot represents a topic. The larger the bubble, the more prevalent is that topic.
##### 2. A good topic model will have fairly big, non-overlapping bubbles scattered throughout the chart instead of being clustered in one quadrant.
##### 3. A model with too many topics, will typically have many overlaps, small sized bubbles clustered in one region of the chart.
##### 4. If you move the cursor over one of the bubbles, the words and bars on the right-hand side will update. These words are the salient keywords that form the selected topic.